## Vector Search on PostgreSQL


### Prerequisites
  
- Generate embeddings - [generate_embeddings.ipynb](../common/generate_embeddings.ipynb)
- Create table and ingest embeddings - [postgree_ingestion.ipynb](.../postgree_ingestion.ipynb)

#### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

pg_host  = os.getenv("POSTGRESQL_HOST")
if pg_host is None or pg_host == "":
    print("POSTGRESQL_HOST environment variable not set.")
    exit()

pg_user  = os.getenv("POSTGRESQL_USERNAME")
if pg_user is None or pg_user == "":
    print("POSTGRESQL_USERNAME environment variable not set.")
    exit()

pg_password  = os.getenv("POSTGRESQL_PASSWORD")
if pg_password is None or pg_password == "":
    print("POSTGRESQL_PASSWORD environment variable not set.")
    exit()

db_name  = os.getenv("POSTGRESQL_DATABASE")
if db_name is None or db_name == "":
    print("POSTGRESQL_DATABASE environment variable not set.")
    exit()

aoai_key  = os.getenv("AZURE_OPENAI_KEY")
if aoai_key is None or aoai_key == "":
    print("AZURE_OPENAI_KEY environment variable not set.")
    exit()

com_vision_key  = os.getenv("COMPUTER_VISION_KEY")
if com_vision_key is None or com_vision_key == "":
    print("COMPUTER_VISION_KEY environment variable not set.")
    exit()

aoai_endpoint = 'https://azure-openai-dnai.openai.azure.com'
aoai_api_version = '2023-08-01-preview'
aoai_embedding_deployed_model = 'embedding-ada'
com_vision_endpoint = 'https://comvis007.cognitiveservices.azure.com/'

text_table_name = 'text_sample'
doc_table_name = 'doc_sample'
image_table_name = 'image_sample'

openai.api_type = "azure"
openai.api_key = aoai_key
openai.api_base = aoai_endpoint
openai.api_version = aoai_api_version

postgresql_params = {
    "host": pg_host,
    "port": "5432", 
    "dbname": db_name,
    "user": pg_user,
    "password": pg_password
}

#### Helper method

In [ ]:
import requests

def vectorize_text_com_vision(com_vision_endpoint,com_vision_key,query):
    vectorize_text_url = f"{com_vision_endpoint}/computervision/retrieval:vectorizeText"  
    params = {  
        "api-version": "2023-02-01-preview"  
    } 
    headers = {  
        "Content-Type": "application/json",  
        "Ocp-Apim-Subscription-Key": com_vision_key  
    }  
    data = {
        'text':query
    }

    response = requests.post(vectorize_text_url, params=params, headers=headers, json=data)
    query_vector = response.json()["vector"]

    return query_vector

def show_image(image_folder, image):
    image_path = os.path.join(image_folder, image)
    plt.imshow(Image.open(image_path))
    plt.axis('off')
    plt.show()

#### Simple vector search

This demo shows how to apply vector search on single columns.

In [ ]:
from psycopg2 import connect
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity

query = 'web hosting services'
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        # A higher value of probes provides better recall at the cost of speed.
        query_sql = f"SET ivfflat.probes = 10;"
        cursor.execute(query_sql)

        # Postgres supports L2 distance (<->), inner product (<#>) and cosine distance (<=>)
        query_sql = f"SELECT title FROM text_sample ORDER BY ((content_vector <=> '{query_vector}')) LIMIT 5;"
        cursor.execute(query_sql)
        records = cursor.fetchall()

        for row in records:
                print(row[0], )

#### Metadata filtering with vector search

This demo shows how to apply metadata filtering (SQL - where, order by etc.) on top of vector search.

In [ ]:
#  TODO

#### Cross column vector search

This demo shows how to apply vector search on multiple columns.

In [ ]:
query = 'tools for software development'
query_vector = get_embedding(query,   engine=aoai_embedding_deployed_model )

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        # A higher value of probes provides better recall at the cost of speed.
        query_sql = f"SET ivfflat.probes = 10;"
        cursor.execute(query_sql)

        query_sql = f'''
            (
                SELECT title, ((title_vector <-> '{query_vector}')) AS content_similarity FROM text_sample
                union
                SELECT title, ((content_vector <-> '{query_vector}')) AS content_similarity FROM text_sample
            ) ORDER BY content_similarity LIMIT 5;
        '''
        cursor.execute(query_sql)
        records = cursor.fetchall()

        for row in records:
                print(row[0])

#### Hybrid search

- This demo shows how to apply vector search in in conjunction with additional search methods, such as lexical search. 

- Implement a hybrid search that combines semantic keyword search by reranking. Details - https://github.com/pgvector/pgvector-python/blob/master/examples/hybrid_search.py

In [ ]:
import itertools
from sentence_transformers import CrossEncoder, SentenceTransformer

query = 'database'

def semantic_search(query):
    query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

    with connect(**postgresql_params) as connection:
        with connection.cursor() as cursor:
            # A higher value of probes provides better recall at the cost of speed.
            query_sql = f"SET ivfflat.probes = 10;"
            cursor.execute(query_sql)

            # Postgres supports L2 distance (<->), inner product (<#>) and cosine distance (<=>)
            query_sql = f"SELECT title FROM text_sample ORDER BY ((content_vector <=> '{query_vector}')) LIMIT 5;"
            cursor.execute(query_sql)
            return cursor.fetchall()

def keyword_search(query):
    with connect(**postgresql_params) as connection:
        with connection.cursor() as cursor:
            cursor.execute("SELECT title FROM text_sample, plainto_tsquery('english', %s) query WHERE to_tsvector('english', content) @@ query ORDER BY ts_rank_cd(to_tsvector('english', content), query) DESC LIMIT 5", (query,))
            return cursor.fetchall()

def rerank(query, records):
    # deduplicate
    results = set(itertools.chain(*records))

    # re-rank
    encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    scores = encoder.predict([(query, item[1]) for item in results])
    return [v for _, v in sorted(zip(scores, results), reverse=True)]

keyword_search_records = keyword_search(query)
semantic_search_records = semantic_search(query)

records = list(semantic_search_records) + (list(keyword_search_records))
results = rerank(query, records)

print(results)

#### Document search example

- This demo shows how to apply vector search for srarching within documents.

In [ ]:
query = 'web hosting services'
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        # A higher value of probes provides better recall at the cost of speed.
        query_sql = f"SET ivfflat.probes = 10;"
        cursor.execute(query_sql)

        # Postgres supports L2 distance (<->), inner product (<#>) and cosine distance (<=>)
        query_sql = f"SELECT chunk_content FROM doc_sample ORDER BY ((chunk_content_vector <=> '{query_vector}')) LIMIT 5;"
        cursor.execute(query_sql)
        records = cursor.fetchall()

        for row in records:
                print(row[0], )

#### Image search example

This demo shows how to apply vector search for searching images.

In [ ]:
from psycopg2 import connect

query = 'flower with hand'
image_folder = "../data/images"
query_vector = vectorize_text_com_vision(com_vision_endpoint,com_vision_key,query)

with connect(**postgresql_params) as connection:
    with connection.cursor() as cursor:
        # A higher value of probes provides better recall at the cost of speed.
        query_sql = f"SET ivfflat.probes = 10;"
        cursor.execute(query_sql)

        # Postgres supports L2 distance (<->), inner product (<#>) and cosine distance (<=>)
        query_sql = f"SELECT image FROM image_sample ORDER BY ((image_vector <=> '{query_vector}')) LIMIT 5;"
        cursor.execute(query_sql)
        records = cursor.fetchall()

        for result in records:
            show_image(image_folder, result[0])
            print("\n")